# Calcul de la consommation globale

On commence par charger les données, on les affiche, on les trie et on fait une première synthèse des consommation par moteur et par vol.

L'idées est de construire une table résumant les données de chaque vol et la consommation correspondate de chaque moteur.

In [1]:
import os
from glob import glob
import pandas as pd
import statsmodels.formula.api as smf
import tabata as tbt
from tabata import Opset
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy import stats
import seaborn as sns
%matplotlib inline
from utils import detect_phase, get_consumption, plot_residus
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
%reload_ext autoreload
%autoreload 2

## 1. Récupération et affichage des données.

On affiche les données de manière à mieux comprendre comment définir les phases de vol.
Pour simplifier les accès j'utilise l'Opset de tabata.

In [3]:
datadir = "../../data/"
datafiles = glob(os.path.join(datadir,'*.h5'))
datafiles

['../../data\\Aircraft_01.h5',
 '../../data\\Aircraft_02.h5',
 '../../data\\Aircraft_03.h5']

In [4]:
sorting = lambda name: int(name[8:])
ds1 = Opset(datafiles[0], sortkey=sorting)
# ds1.plotc()

In [5]:
ds2 = Opset(datafiles[1], sortkey=sorting)
ds2.storename

'../../data\\Aircraft_02.h5'

In [6]:
ds3 = Opset(datafiles[2], sortkey=sorting)
ds3.storename

'../../data\\Aircraft_03.h5'

### Recherche d'un moyen d'identifier les phases de vol

Un affichage interactif avec des variables en parallèle de l'altitude.

In [13]:
ds1[0][['TLA_1 [deg]', 'TLA_2 [deg]']].describe()

,TLA_1 [deg],TLA_2 [deg]
count,7429.000000,7429.000000
mean,20.223873,20.251384
std,14.641715,14.679342
min,-24.946059,-24.946059
25%,0.509813,0.614092
50%,29.221533,29.221533
75%,30.044186,30.044186
max,55.905590,55.905590


In [7]:
from ipywidgets import interact, widgets

def mydoubleplot(varname, record):
    df = ds1[record]
    tbt.doubleplot(df[varname], df["N2_1 [% rpm]"], title="Vol {}".format(record))

interact(mydoubleplot, varname=ds1.df.columns, record=widgets.IntSlider(0,0,len(ds1)-1,1)) ; #range(len(ds))) ;

interactive(children=(Dropdown(description='varname', options=('ALT [ft]', 'EGT_1 [deg C]', 'EGT_2 [deg C]', '…

In [10]:
all_phases = ['taxi1','climb','cruise','descend','taxi2']
dfc1_all = [get_consumption(ds1, phase=p) for p in all_phases]
dfc2_all = [get_consumption(ds2, phase=p) for p in all_phases]
dfc3_all = [get_consumption(ds3, phase=p) for p in all_phases]

In [11]:
df_fin = pd.concat([d for d in dfc1_all] + [d for d in dfc2_all] + [d for d in dfc3_all], axis=0, ignore_index=True)
df_fin

,AC,ENG,Flight,Phase,Leg_flight_H,Leg_phase_H,Alt_max_Ft,Alt_slope,Avg_egt,TAT_max,TAT_min,T_oil_range,M_max,Volume_L,Weight_Kg
0,AC01,Left,0,taxi1,2.063611,0.228611,39439.264501,0.640898,0.039437,862.015543,40.814093,-22.163944,59.739683,75.880634,116.610233
1,AC01,Right,0,taxi1,2.063611,0.228611,39439.264501,0.640898,0.039437,862.015543,40.814093,-22.163944,62.940023,83.859412,116.610233
2,AC01,Left,1,taxi1,2.071944,0.176667,39457.811063,0.649114,0.029161,816.434746,40.543801,-20.271900,53.339003,52.222420,82.690332
3,AC01,Right,1,taxi1,2.071944,0.176667,39457.811063,0.649114,0.029161,816.434746,40.543801,-20.271900,54.939173,61.052008,82.690332
4,AC01,Left,2,taxi1,1.956111,0.264444,40825.620025,0.624464,0.029223,1022.816007,32.164749,-29.191536,68.807313,126.534512,191.529825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29915,AC03,Right,999,taxi2,1.458611,0.127778,37621.701406,0.591598,0.050398,862.242237,14.325476,-32.164749,13.334751,40.471067,57.637088
29916,AC03,Left,1000,taxi2,1.976944,0.100556,36471.814551,0.632681,0.076850,1106.465433,14.866060,-25.677740,14.401531,51.539080,76.348430
29917,AC03,Right,1000,taxi2,1.976944,0.100556,36471.814551,0.632681,0.076850,1106.465433,14.866060,-25.677740,14.934921,53.047811,76.348430
29918,AC03,Left,1001,taxi2,1.505000,0.141667,37134.854149,0.599814,0.063640,932.233596,12.433432,-32.705333,13.334751,46.703366,69.880999


## Mise en place du modèle multiphase

In [12]:
import plotly.express as px

def shapiro_test(res):
    residus = res.resid
    test = shapiro(residus)
    print(f"Statistique de test:      {test.statistic}\n")
    print(f"P-value:                   {test.pvalue}\n")
    if test.pvalue < 0.05:
        print("Les résidus ne sont pas gaussiens.\n\n\n\n")
    else:
        print("Les résidus sont gaussiens.\n\n\n\n")

def plot_predictions(dataframe, results, phase):
    df = dataframe.copy()
    df["Prediction"] = results.predict()
    fig = px.scatter(df, x="Prediction", y="Volume_L", color="AC", symbol="ENG", 
           hover_data=dict(AC=True, ENG=True, Flight=True), 
           opacity=0.4,
           title="Consommation en "+phase.upper(),
           labels={"Volume_L" : "Consommation (litres)", "Prediction" : "Estimation de la consommation"})
    fig.update_traces(marker=dict(size=12))
    fig.show()

In [13]:
features = ["Leg_flight_H", "Leg_phase_H", "Alt_max_Ft", "Alt_slope", "Avg_egt", "TAT_max", "TAT_min", "T_oil_range", "M_max"]

In [14]:
class ModelConsoByPhases:
    """
    Un modèle réalisant un apprentissage sur une série de données et proposant une fonction de 
        Paramètres: 
            ac, acs: One aircraft flights or a liste of aircraft flights.
            phases: list of phases (None for all phases)
            altitude_threshold: threshold for extracting cruising phase 
            width: un paramètre correspondant à la largeur d'un filtrage pour le calcul de l'erreur relative 
        Méthodes:
            __init__(altitude_threshold) : creation du modèle.
            fit(acs, phases) : apprentissage.
            predict(ac) : calcul de la consommation estimée.
            score(ac, width) : estimation de l'erreur relative moyenne pour un avion.
    """
    
    def __init__(self, acs, phases = None, altitude_threshold = 0.95):
        """
            phases: a phase or a list of phases between taxi1, climb, cruise, descend, taxi2.
            ac, acs: one aircraft flight or a liste of aircraft flights.
        """
        if isinstance(phases,str):
            if phases=='all':
                phases = ['taxi1','climb','cruise','descend','taxi2']
            else:
                phases = [phases]
                
        assert len(phases)>=1, "At least one phase should be given"

        self.aircrafts = acs
        # self.aircraft_names = [ac.storename[-14:-3] for ac in acs]
        self.phases = phases
        self.threshold = altitude_threshold
        self.final_dataset = self.construct_dataset()
        self.variables = {p:features  for p in self.phases}
        self.models = self.add_models(self.variables)
        self.results = {}
        self.predictions = {}
        self.accuracy = {}
        


    def construct_dataset(self):
        if len(self.aircrafts) == 1:
            return self.aircrafts[0]
        else:
            all_ = []
            for ac in self.aircrafts:
                for p in self.phases:
                    all_.append(get_consumption(ac, phase=p, threshold=self.threshold))
            return pd.concat([d for d in all_], axis=0, ignore_index=True).dropna()

    def add_models(self, var):
        var = self.variables
        df = self.final_dataset
        dfs = { p:df[df['Phase'] == p] for p in self.phases }
        models = {p: smf.ols(f"Volume_L ~ {'+'.join(var[p])}", data=dfs[p])
                   for p in self.phases}
        return models
    
    def fit(self):
        for p, mod in self.models.items():
            self.results[p] = mod.fit()

    def update_features(self):
        results = self.results
        for p in results:
            pvalues = results[p].pvalues
            self.variables[p] = pvalues[pvalues<0.05].index[1:]
        self.models = self.add_models(self.variables)
        self.fit()

    def predict(self, data):
        for p in self.phases:
            self.predictions[p] = self.results[p].predict(data)  
            self.accuracy[p] = accuracy_score(data, self.results[p].predict(data))

        
    def print_results(self, phase, summary=True, analyse_residus=False, prediction_plot=False):
        df = self.final_dataset[ self.final_dataset['Phase']==phase ]
        results = self.results[phase]
        if summary:
            return results.summary()
        if analyse_residus:
            shapiro_test(results)
            plot_residus(results, title=phase.upper())
        if prediction_plot:
            plot_predictions(df, results, phase)

Why
        

In [15]:
# X_train, X_test, y_train, y_test = train_test_split(df_fin, df_fin['Volume_L'], test_size=0.2, random_state=42)

In [16]:
# model = ModelConsoByPhases(acs= [ds1, ds2], phases=['climb', 'descend'])
# model = ModelConsoByPhases(acs= [df_fin], phases=['climb', 'descend'])
model = ModelConsoByPhases(acs= [df_fin], phases='all')

In [17]:
model.final_dataset.head()

,AC,ENG,Flight,Phase,Leg_flight_H,Leg_phase_H,Alt_max_Ft,Alt_slope,Avg_egt,TAT_max,TAT_min,T_oil_range,M_max,Volume_L,Weight_Kg
0,AC01,Left,0,taxi1,2.063611,0.228611,39439.264501,0.640898,0.039437,862.015543,40.814093,-22.163944,59.739683,75.880634,116.610233
1,AC01,Right,0,taxi1,2.063611,0.228611,39439.264501,0.640898,0.039437,862.015543,40.814093,-22.163944,62.940023,83.859412,116.610233
2,AC01,Left,1,taxi1,2.071944,0.176667,39457.811063,0.649114,0.029161,816.434746,40.543801,-20.271900,53.339003,52.222420,82.690332
3,AC01,Right,1,taxi1,2.071944,0.176667,39457.811063,0.649114,0.029161,816.434746,40.543801,-20.271900,54.939173,61.052008,82.690332
4,AC01,Left,2,taxi1,1.956111,0.264444,40825.620025,0.624464,0.029223,1022.816007,32.164749,-29.191536,68.807313,126.534512,191.529825


In [18]:
# data = model.final_dataset
# data[data['Flight']==0]

In [19]:
print(model.models)
model.fit()
print(model.results)

{'taxi1': <statsmodels.regression.linear_model.OLS object at 0x00000286F8246390>, 'climb': <statsmodels.regression.linear_model.OLS object at 0x00000286CC21B610>, 'cruise': <statsmodels.regression.linear_model.OLS object at 0x00000286CBDBFA10>, 'descend': <statsmodels.regression.linear_model.OLS object at 0x00000286CBDCE690>, 'taxi2': <statsmodels.regression.linear_model.OLS object at 0x00000286CD54D710>}
{'taxi1': <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000286CD4B0ED0>, 'climb': <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000286CD51D910>, 'cruise': <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000286CBDC1850>, 'descend': <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000286CDA5BC50>, 'taxi2': <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x00000286CDA5BFD0>}


In [20]:
# model.update_features()

In [21]:
model.print_results('taxi2', summary=True, analyse_residus=False, prediction_plot=False)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Volume_L   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     1066.
Date:                Fri, 02 Feb 2024   Prob (F-statistic):               0.00
Time:                        10:09:42   Log-Likelihood:                -24441.
No. Observations:                5984   AIC:                         4.890e+04
Df Residuals:                    5977   BIC:                         4.894e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     -73.4485      2.290    -32.077      0.000     -77.937     -68.960
Leg_phase_H   251.1084      3.527     71.190      0.000     244.194     258.023
Avg_egt         2.4806      0.412      6.015      0.000       1.672       3.289
TAT_max         0.0973      0.002     54.153      0.000       0.094       0.101
TAT_min        -0.4108      0.022    -18.979      0.000      -0.453      -0.368
T_oil_range    -0.0612      0.033     -1.835      0.067      -0.127       0.004
M_max           0.1267      0.012     10.217      0.000       0.102       0.151
==============================================================================
Omnibus:                    12468.516   Durbin-Watson:                   1.234
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        164717704.385
Skew:                          17.089   Prob(JB):                         0.00
Kurtosis:                     815.074   Cond. No.                     1.76e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [324]:
class ModelConsoByPhases:
    """
    Un modèle réalisant un apprentissage sur une série de données et proposant une fonction de 
        Paramètres: 
            ac, acs: One aircraft flights or a liste of aircraft flights.
            phases: list of phases (None for all phases)
            altitude_threshold: threshold for extracting cruising phase 
            width: un paramètre correspondant à la largeur d'un filtrage pour le calcul de l'erreur relative 
        Méthodes:
            __init__(altitude_threshold) : creation du modèle.
            fit(acs, phases) : apprentissage.
            predict(ac) : calcul de la consommation estimée.
            score(ac, width) : estimation de l'erreur relative moyenne pour un avion.
    """
    
    def __init__(self, acs, phases = None, altitude_threshold = 0.95):
        """
            phases: a phase or a list of phases between taxi1, climb, cruise, descend, taxi2.
            ac, acs: one aircraft flight or a liste of aircraft flights.
        """
        if isinstance(phases,str):
            phases = [phases]
        assert len(phases)>=1, "At least one phase should be given"

        self.aircrafts = acs
        self.aircraft_names = [ac.storename[-14:-3] for ac in acs]
        self.phases = phases
        self.altitude_threshold = altitude_threshold
        self.models = [ [None]*len(self.phases) ]*len(self.aircrafts)
        self.results = [ [None]*len(self.phases) ]*len(self.aircrafts)


        # Creation of models
        for i, ac in enumerate(self.aircrafts):
            for j, phase in enumerate(self.phases):
                # print(j, phase)
                ds.rewind()
                df_phase = get_consumption(ac, phase)
                model = smf.ols("Consumption ~ Duration + Alt_max + Mach_max", data=df_phase)
                self.models[i][j] = model


    def fit(self):
        """
            Apprentissage des modèles
        """
        for i, ac in enumerate(self.aircrafts):
            for j, phase in enumerate(self.phases):
                self.results[i][j] =  self.models[i][j].fit()

    def print_results(self, ac, phase, summary=True, shapiro=False, analyse_resid=False, predict_plot=False):
        name = ac.storename[-14:-3]
        i, j = self.aircraft_names.index(name), self.phases.index(phase)
        res = self.results[i][j]
        print(f"------------------------{ac.storename[-14:-3].upper()}------------------------")
        print(f"------------------------{phase.upper()}------------------------")
        if summary: print(f"{res.summary()}\n\n\n")
        if shapiro: shapiro_test(res)
        if analyse_resid: plot_residus(res)
        # if predict_plot: predictions_plot() 
            
        

